In [1]:
from datetime import date
import datetime
import pandas as pd

pd.set_option('display.max_columns', None)

today = date.today()
week = datetime.timedelta(7)
week_ago = today - week
two_month = datetime.timedelta(59)
two_month_ago = today - two_month
years = datetime.timedelta(729)
years_ago = today - years

In [2]:
import yfinance as yf

spy = yf.Ticker("spy")
spy_1 = spy.history(start=week_ago, end=today, interval="1m")
spy_1 = spy_1.drop(columns=["Dividends", "Stock Splits"])
spy_1.reset_index(inplace = True, drop = False)
spy_5 = spy.history(start=two_month_ago, end=today, interval="5m")
spy_5 = spy_5.drop(columns=["Dividends", "Stock Splits"])
spy_5.reset_index(inplace = True, drop = False)
spy_15 = spy.history(start=two_month_ago, end=today, interval="15m")
spy_15 = spy_15.drop(columns=["Dividends", "Stock Splits"])
spy_15.reset_index(inplace = True, drop = False)
spy_30 = spy.history(start=two_month_ago, end=today, interval="30m")
spy_30 = spy_30.drop(columns=["Dividends", "Stock Splits"])
spy_30.reset_index(inplace = True, drop = False)
spy_60 = spy.history(start=years_ago, end=today, interval="1h")
spy_60 = spy_60.drop(columns=["Dividends", "Stock Splits"])
spy_60.reset_index(inplace = True, drop = False)

- SPY: Invalid input - interval=4h is not supported. Valid intervals: , 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]


In [3]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()    

    up_chg = 0 * diff
    down_chg = 0 * diff
    
    up_chg[diff > 0] = diff[ diff>0 ]
    
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [4]:
spy_1['RSI'] = computeRSI(spy_1['Close'], 14)
spy_5['RSI'] = computeRSI(spy_5['Close'], 14)
spy_15['RSI'] = computeRSI(spy_15['Close'], 14)
spy_30['RSI'] = computeRSI(spy_30['Close'], 14)
spy_60['RSI'] = computeRSI(spy_60['Close'], 14)

In [5]:
from scipy.stats import zscore
import numpy as np

def calculate_zscore(dataframe):
    dataframe['RSI zscore'] = zscore(dataframe['RSI'], nan_policy='omit')

In [6]:
frames = [spy_1, spy_5, spy_15, spy_30, spy_60]
for frame in frames:
    calculate_zscore(frame)

In [7]:
def calculate_returns(length, column_name, dataframe):
    
    for ind in dataframe.index:
        if ind >= len(dataframe) - length:
            dataframe[column_name][ind] = 0
        else:
            dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100



In [8]:
spy_1["5 m Return"] = np.nan
spy_1["15 m Return"] = np.nan
spy_1["30 m Return"] = np.nan
spy_1["Hourly Return"] = np.nan
spy_1["2 Hour Return"] = np.nan
spy_1["4 Hour Return"] = np.nan

calculate_returns(5, "5 m Return", spy_1)
calculate_returns(15, "15 m Return", spy_1)
calculate_returns(30, "30 m Return", spy_1)
calculate_returns(60, "Hourly Return", spy_1)
calculate_returns(120, "2 Hour Return", spy_1)
calculate_returns(240, "4 Hour Return", spy_1)
spy_1 = spy_1.sort_values(by=['RSI'])
spy_1 = spy_1.dropna()
spy_1

<ipython-input-7-b7139c319bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100
<ipython-input-7-b7139c319bc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = 0


,Datetime,Open,High,Low,Close,Volume,RSI,RSI zscore,5 m Return,15 m Return,30 m Return,Hourly Return,2 Hour Return,4 Hour Return
1124,2022-01-31 15:14:00-05:00,446.445007,446.549988,445.970001,446.160004,517843,14.401251,-3.299498,-0.041464,0.237582,0.439302,0.658957,0.484132,0.784472
684,2022-01-28 14:24:00-05:00,433.059998,433.209991,432.390015,432.399994,379319,15.058707,-3.243111,0.043941,0.185018,0.629047,1.207216,2.402871,3.307127
1123,2022-01-31 15:13:00-05:00,446.711609,446.714996,446.290009,446.440002,363563,16.597053,-3.111176,-0.103035,0.179193,0.409906,0.562228,0.394232,0.698861
1129,2022-01-31 15:19:00-05:00,445.980011,445.994995,445.690002,445.975006,515104,18.495736,-2.948336,0.280285,0.548239,0.673806,0.783675,0.528052,0.819552
1128,2022-01-31 15:18:00-05:00,446.109985,446.130005,445.750000,445.980011,277377,18.543539,-2.944236,0.300461,0.500018,0.614327,0.679402,0.466386,0.773573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,2022-01-31 11:55:00-05:00,446.700012,446.799988,446.660004,446.700012,236269,80.542744,2.373105,-0.024629,-0.118648,-0.182450,0.145510,0.114166,0.696213
924,2022-01-31 11:54:00-05:00,446.630005,446.720001,446.500000,446.700012,202347,80.542744,2.373105,0.002234,-0.145517,-0.149992,0.118648,0.082829,0.749939
770,2022-01-28 15:50:00-05:00,439.890015,440.820007,439.890015,440.600006,2222548,80.600174,2.378031,-0.013617,0.304109,0.229228,0.923742,0.887427,1.529730
926,2022-01-31 11:56:00-05:00,446.700012,446.769989,446.630005,446.710114,320035,80.647566,2.382095,-0.040320,-0.123147,-0.125388,0.167867,0.120858,0.736470


In [9]:
spy_5["5 m Return"] = np.nan
spy_5["15 m Return"] = np.nan
spy_5["30 m Return"] = np.nan
spy_5["Hourly Return"] = np.nan
spy_5["2 Hour Return"] = np.nan
spy_5["4 Hour Return"] = np.nan

calculate_returns(1, "5 m Return", spy_5)
calculate_returns(3, "15 m Return", spy_5)
calculate_returns(6, "30 m Return", spy_5)
calculate_returns(12, "Hourly Return", spy_5)
calculate_returns(24, "2 Hour Return", spy_5)
calculate_returns(48, "4 Hour Return", spy_5)
spy_5 = spy_5.sort_values(by=['RSI'])
spy_5 = spy_5.dropna()
spy_5

<ipython-input-7-b7139c319bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100
<ipython-input-7-b7139c319bc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = 0


,Datetime,Open,High,Low,Close,Volume,RSI,RSI zscore,5 m Return,15 m Return,30 m Return,Hourly Return,2 Hour Return,4 Hour Return
2498,2022-01-21 09:40:00-05:00,445.100006,445.279999,443.630005,443.730011,3645551,7.464443,-3.277822,0.303381,-0.022538,-0.394384,0.216325,0.331283,-0.112681
2497,2022-01-21 09:35:00-05:00,446.429993,446.559998,445.000000,445.100006,2995122,8.889696,-3.171387,-0.307795,-0.314535,-0.460575,-0.188721,0.013658,-0.458326
1878,2022-01-10 10:00:00-05:00,459.339996,459.420013,458.459991,458.470001,2215195,9.286220,-3.141775,0.194120,-0.226843,-0.087245,-0.145050,0.034899,0.896457
1877,2022-01-10 09:55:00-05:00,459.614990,459.779999,459.010010,459.359985,1679958,10.522692,-3.049438,-0.193744,-0.372255,-0.257967,-0.341775,0.026129,0.548594
1700,2022-01-05 14:40:00-05:00,472.339996,472.359985,471.309998,471.369995,2437253,10.648028,-3.040078,0.116685,0.067908,-0.108217,-0.224876,-1.054354,-0.445511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2021-12-07 09:55:00-05:00,466.709991,467.260010,466.649994,467.239990,1231338,83.770083,2.420540,-0.034264,0.153973,0.057790,0.262395,0.188341,0.261108
87,2021-12-07 10:15:00-05:00,467.970001,468.070007,467.769989,467.957092,1267423,84.030991,2.440024,-0.061347,-0.027162,0.086096,0.081825,0.015580,0.062593
86,2021-12-07 10:10:00-05:00,467.420013,467.975006,467.339996,467.959412,1955036,84.057129,2.441976,-0.000496,-0.096034,0.079190,0.046178,0.079190,0.066368
1014,2021-12-23 09:30:00-05:00,468.750000,469.750000,468.640015,469.679993,2265141,85.802748,2.572335,0.159898,0.057490,0.085163,0.166070,0.300205,0.300205


In [10]:
spy_15["15 m Return"] = np.nan
spy_15["30 m Return"] = np.nan
spy_15["Hourly Return"] = np.nan
spy_15["2 Hour Return"] = np.nan
spy_15["4 Hour Return"] = np.nan

calculate_returns(1, "15 m Return", spy_15)
calculate_returns(2, "30 m Return", spy_15)
calculate_returns(4, "Hourly Return", spy_15)
calculate_returns(8, "2 Hour Return", spy_15)
calculate_returns(16, "4 Hour Return", spy_15)
spy_15 = spy_15.sort_values(by=['RSI'])
spy_15 = spy_15.dropna()
spy_15

<ipython-input-7-b7139c319bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100
<ipython-input-7-b7139c319bc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = 0


,Datetime,Open,High,Low,Close,Volume,RSI,RSI zscore,15 m Return,30 m Return,Hourly Return,2 Hour Return,4 Hour Return
834,2022-01-21 10:00:00-05:00,443.640015,444.309998,441.590393,441.980011,9341126,11.704888,-2.634782,0.185524,0.613127,0.889179,0.391416,0.113100
833,2022-01-21 09:45:00-05:00,443.739990,445.700012,443.140015,443.630005,9009593,13.262057,-2.531705,-0.371930,-0.187097,0.545496,0.197236,-0.326735
832,2022-01-21 09:30:00-05:00,445.559998,446.670013,443.630005,443.730011,14041447,13.362104,-2.525083,-0.022538,-0.394384,0.216325,0.331283,-0.112681
626,2022-01-10 10:00:00-05:00,459.339996,459.579987,457.589996,457.649994,6250355,14.369356,-2.458408,0.114715,-0.109254,0.281877,0.220695,1.090357
571,2022-01-05 15:45:00-05:00,470.320007,470.350006,468.290009,468.380005,13696770,15.478812,-2.384968,0.339467,-0.283961,-0.153722,0.046990,0.053375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2021-12-27 13:45:00-05:00,476.019989,476.309998,475.929993,476.299988,1373442,83.434422,2.113338,-0.006298,0.061939,-0.027288,0.203653,0.268738
340,2021-12-23 10:00:00-05:00,470.179993,470.815002,470.000000,470.708496,1594262,83.533919,2.119924,-0.016675,-0.031548,0.113977,0.078922,0.144786
344,2021-12-23 11:00:00-05:00,471.079987,471.470001,470.910004,471.244995,1916717,83.595412,2.123995,-0.068962,-0.030767,-0.035015,-0.092308,0.087004
383,2021-12-27 14:15:00-05:00,476.260010,476.600006,476.200012,476.595001,1329505,84.327580,2.172460,-0.080783,-0.089172,-0.068195,0.393416,0.305288


In [11]:
spy_30["30 m Return"] = np.nan
spy_30["Hourly Return"] = np.nan
spy_30["2 Hour Return"] = np.nan
spy_30["4 Hour Return"] = np.nan

calculate_returns(1, "30 m Return", spy_30)
calculate_returns(2, "Hourly Return", spy_30)
calculate_returns(4, "2 Hour Return", spy_30)
calculate_returns(8, "4 Hour Return", spy_30)
spy_30 = spy_30.sort_values(by=['RSI'])
spy_30 = spy_30.dropna()
spy_30

<ipython-input-7-b7139c319bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100
<ipython-input-7-b7139c319bc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = 0


,Datetime,Open,High,Low,Close,Volume,RSI,RSI zscore,30 m Return,Hourly Return,2 Hour Return,4 Hour Return
434,2022-01-24 12:00:00-05:00,425.820007,425.820007,420.760010,422.579987,15012133,11.651278,-2.444248,0.643665,0.880307,1.398553,1.550007
433,2022-01-24 11:30:00-05:00,427.739990,428.049988,422.890015,425.799988,14677095,13.437003,-2.334984,-0.756224,-0.117426,1.282297,3.311415
432,2022-01-24 11:00:00-05:00,428.779999,428.824005,426.459991,427.720001,12066529,14.683007,-2.258744,-0.448895,-1.201724,-0.331996,1.246139
430,2022-01-24 10:00:00-05:00,429.089996,430.339996,425.750000,428.660004,21673844,14.939387,-2.243057,0.030328,-0.219289,-1.418378,-0.039662
429,2022-01-24 09:30:00-05:00,431.190002,433.459991,428.459991,429.070007,34424758,15.193864,-2.227486,-0.095556,-0.065257,-0.762118,0.510407
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021-12-28 09:30:00-05:00,477.720001,478.809998,477.230011,478.470001,5602638,84.767534,2.029547,-0.114947,-0.131671,-0.087783,-0.271697
19,2021-12-07 12:30:00-05:00,468.100006,468.641693,468.089996,468.616791,2205551,85.155081,2.053260,-0.091074,-0.033460,-0.093210,-0.248984
16,2021-12-07 11:00:00-05:00,468.459991,468.880005,468.040009,468.295013,5781427,85.873719,2.097232,-0.037374,-0.041642,-0.022424,-0.218884
14,2021-12-07 10:00:00-05:00,467.239990,468.149994,466.963196,467.510010,8905002,86.074249,2.109501,0.204486,0.167912,0.126200,0.173258


In [12]:
spy_60["Hourly Return"] = np.nan
spy_60["2 Hour Return"] = np.nan
spy_60["4 Hour Return"] = np.nan

calculate_returns(1, "Hourly Return", spy_60)
calculate_returns(2, "2 Hour Return", spy_60)
calculate_returns(4, "4 Hour Return", spy_60)
spy_60 = spy_60.sort_values(by=['RSI'])
spy_60 = spy_60.dropna()
spy_60

<ipython-input-7-b7139c319bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = ((dataframe['Close'][ind+length] - dataframe['Close'][ind]) /(dataframe['Close'][ind])) * 100
<ipython-input-7-b7139c319bc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name][ind] = 0


,index,Open,High,Low,Close,Volume,RSI,RSI zscore,Hourly Return,2 Hour Return,4 Hour Return
95,2020-02-25 13:30:00-05:00,316.600006,316.679993,312.549988,312.609985,35484094,10.614362,-3.269740,0.172742,0.028798,1.570648
97,2020-02-25 15:30:00-05:00,313.144989,314.070007,311.690002,312.700012,32657742,12.934690,-3.096120,1.164050,1.541406,0.441309
96,2020-02-25 14:30:00-05:00,312.605011,315.660004,312.079987,313.149994,40570081,13.280969,-3.070210,-0.143695,1.018682,0.303373
94,2020-02-25 12:30:00-05:00,318.140015,319.070007,316.170013,316.600006,18112557,13.452533,-3.057372,-1.260272,-1.089707,-0.082126
87,2020-02-24 12:30:00-05:00,322.609985,323.440002,321.239990,322.559998,16969846,13.668620,-3.041204,0.325517,0.455730,-0.713042
...,...,...,...,...,...,...,...,...,...,...,...
2481,2021-07-02 11:30:00-04:00,432.109985,432.649994,432.040009,432.500000,4406688,88.658831,2.569986,0.152581,0.244183,0.289017
2482,2021-07-02 12:30:00-04:00,432.489990,433.489990,432.459991,433.159912,6475580,90.099224,2.677764,0.091463,0.101601,-0.205449
2483,2021-07-02 13:30:00-04:00,433.160004,433.559998,433.029999,433.556091,4088361,90.850519,2.733980,0.010129,0.044725,-0.557270
2484,2021-07-02 14:30:00-04:00,433.559998,433.670013,433.179993,433.600006,6850584,90.932653,2.740126,0.034593,-0.306738,-0.708028
